In [7]:
from common import *

In [8]:
RAW_DIR = ("data", "raw")
OUT_DIR = ("data", "pre")

WINDOW_SIZE = 12
TIME_AVG_BLOCK = 12

In [9]:
def box_position(img):
   hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
   hsv = cv2.blur(hsv, (9,9))

   # create mask
   mask1 = cv2.inRange(hsv, (14,90,57), (24,208,110))

   mask1 = cv2.erode(mask1, np.ones((3, 3), np.uint8), iterations=4)
   mask1 = cv2.dilate(mask1, np.ones((5, 5), np.uint8), iterations=3)


   contours, _ = cv2.findContours(mask1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

   hulls = [cv2.convexHull(cnt) for cnt in contours]
   hull = max(hulls, key=cv2.contourArea)   

   box = np.intp(cv2.boxPoints(cv2.minAreaRect(hull)))

   pos = np.rint(np.sum(box, axis=0) / len(box)).astype(np.int32)

   return pos

def average_by_row(img):
   h, w, _ = img.shape

   for y in range(h):
      c = np.average(img[y], axis=0)

      c = np.rint(c).astype(img.dtype)      
      img[y][:] = c

   return img

In [10]:
def extract_time(stamp):
    t = stamp[stamp.find("@") + 1: stamp.find(".")]
    t = list(map(int, t.split("_")))

    return t[0] * 3600 + t[1]*60 + t[0]

In [11]:
class SlidingAvg:
    def __init__(self, size, initval, dtype):
        self._size = size
        self._arr = np.full([size, *initval.shape], initval, dtype=dtype)
        self._head = 0

    def update(self, val):
        self._arr[self._head] = val
        self._head = (self._head + 1) % self._size

        return np.average(self._arr, axis=0)

class AvgAccumulator:
    def __init__(self, size, shape, dtype, metastrat):
        self._arr = np.ndarray((size, *shape), dtype=dtype)
        self._metas = [None]*size
        self._count = 0
        self._size = size
        self._metastrat = metastrat

    def is_ready(self):
        return self._count == self._size
    
    def get(self):
        avg = np.average(self._arr, axis=0)
        avg = np.rint(avg).astype(self._arr[0].dtype)
        self._count = 0
        return avg, self._metastrat(self._metas)
    
    def add(self, val, meta):
        if self.is_ready():
            raise RuntimeError("Accumulator is full")
        
        self._arr[self._count] = val
        self._metas[self._count] = meta
        self._count += 1


In [12]:
names, times = get_names(RAW_DIR, extract_time)
print(names)
times = np.array(times, dtype=np.int64)
times -= times[0]

img = cv2.imread(names[0])
target_pos = box_position(img)
shape = crop_image(img, CROP_X, CROP_Y).shape
h, w, _ = shape

video = cv2.VideoWriter("out.avi", 0, 60, (w, h))

window = SlidingAvg(WINDOW_SIZE, target_pos, np.int32)
acc = AvgAccumulator(TIME_AVG_BLOCK, shape, img.dtype, lambda m: m[0])

i = 0
for name in tqdm.tqdm(names):
    img = cv2.imread(name)
    time = times[i]

    pos = window.update(box_position(img))
    img = translate_img(img, target_pos - pos)
    
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    img = rotate_image(img, ROTATION)
    img = crop_image(img, CROP_X, CROP_Y)
    img = average_by_row(img)
    
    acc.add(img, time)

    if acc.is_ready():
        img, time = acc.get()   
        
        path = os.path.join(*OUT_DIR, str(time)+".png")
        cv2.imwrite(path, img[:, 0:1])

        video.write(img)

    i += 1

video.release()

['data\\raw\\2025-01-15@16_26_52.png', 'data\\raw\\2025-01-15@16_26_57.png', 'data\\raw\\2025-01-15@16_27_12.png', 'data\\raw\\2025-01-15@16_27_17.png', 'data\\raw\\2025-01-15@16_27_2.png', 'data\\raw\\2025-01-15@16_27_22.png', 'data\\raw\\2025-01-15@16_27_27.png', 'data\\raw\\2025-01-15@16_27_32.png', 'data\\raw\\2025-01-15@16_27_37.png', 'data\\raw\\2025-01-15@16_27_42.png', 'data\\raw\\2025-01-15@16_27_47.png', 'data\\raw\\2025-01-15@16_27_52.png', 'data\\raw\\2025-01-15@16_27_57.png', 'data\\raw\\2025-01-15@16_27_7.png', 'data\\raw\\2025-01-15@16_28_12.png', 'data\\raw\\2025-01-15@16_28_17.png', 'data\\raw\\2025-01-15@16_28_2.png', 'data\\raw\\2025-01-15@16_28_22.png', 'data\\raw\\2025-01-15@16_28_27.png', 'data\\raw\\2025-01-15@16_28_32.png', 'data\\raw\\2025-01-15@16_28_38.png', 'data\\raw\\2025-01-15@16_28_43.png', 'data\\raw\\2025-01-15@16_28_48.png', 'data\\raw\\2025-01-15@16_28_53.png', 'data\\raw\\2025-01-15@16_28_58.png', 'data\\raw\\2025-01-15@16_28_7.png', 'data\\raw\\202

100%|██████████| 4268/4268 [02:51<00:00, 24.83it/s]
